In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/cleaned_santander_data.csv')

/var/folders/jl/5_rcdjz121v7rnhqrmsz6df80000gn/T/ipykernel_1886/1706210101.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/cleaned_santander_data.csv')


## BASELINE MODELLING

In [3]:
df_train[df_train['fecha_dato'] < '2016-05-01']

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12687899,2016-04-28,1297410,N,ES,V,47.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
12687900,2016-04-28,1297411,N,ES,H,41.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
12687901,2016-04-28,1297412,N,ES,V,34.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,1
12687902,2016-04-28,1297392,N,ES,H,44.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [4]:
df_train[df_train['fecha_dato'] >= '2016-05-01']['fecha_dato'].unique()

array(['2016-05-28'], dtype=object)

In [5]:

# Split: train on data before May 2016, validate on May 2016
train_df = df_train[df_train['fecha_dato'] < '2016-05-01']
eval_df = df_train[df_train['fecha_dato'] == '2016-05-28']

In [6]:
product_cols = [col for col in df_train.columns if col.endswith('ult1')]

In [7]:
# Sum of additions per product
product_popularity = train_df[product_cols].sum().sort_values(ascending=False)

# Get top 7 products
top_7_products = product_popularity.head(7).index.tolist()
print(top_7_products)

['ind_cco_fin_ult1', 'ind_ctop_fin_ult1', 'ind_recibo_ult1', 'ind_ecue_fin_ult1', 'ind_cno_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1']


In [9]:
# Get unique customers in May 2016
eval_users = eval_df['ncodpers'].unique()

# Predict same top 7 for all
baseline_preds = pd.DataFrame({
    'ncodpers': eval_users,
    'added_products': [' '.join(top_7_products)] * len(eval_users)
})

In [8]:
def apk(actual, predicted, k=7):
    if len(predicted) > k:
        predicted = predicted[:k]

    if not actual:
        return 0

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual_list, predicted_list, k=7):
    return np.mean([apk(a, p.split(), k) for a, p in zip(actual_list, predicted_list)])


In [10]:
# Ground truth from eval_df
eval_truth = []
for _, row in eval_df.groupby('ncodpers'):
    added = [prod for prod in product_cols if row[prod].values[0] == 1]
    eval_truth.append(added)

# Compute MAP@7
map_score = mapk(eval_truth, baseline_preds['added_products'].tolist())
print(f"Baseline MAP@7: {map_score:.4f}")

Baseline MAP@7: 0.5937


In [16]:
baseline_preds

,ncodpers,added_products
0,657640,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
1,657788,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
2,657795,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
3,657790,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
4,657794,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
...,...,...
931448,1166765,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
931449,1166764,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
931450,1166763,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
931451,1166789,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...


## Baseline Model Error analysis

In [ ]:
# Convert predictions into list of products
baseline_preds['prediction_list'] = baseline_preds['added_products'].apply(lambda x: x.split())

# Create a comparison dataframe
comparison = pd.DataFrame({
    "ncodpers": baseline_preds['ncodpers'],
    "actual": eval_truth,
    "predicted": baseline_preds['prediction_list']
})

In [14]:
#  False Positives & False Negatives
comparison['false_positives'] = comparison.apply(
    lambda row: [p for p in row['predicted'] if p not in row['actual']], axis=1)

comparison['false_negatives'] = comparison.apply(
    lambda row: [a for a in row['actual'] if a not in row['predicted']], axis=1)

In [15]:

# Count most common FP/FN products
fp_counts = pd.Series([p for row in comparison['false_positives'] for p in row]).value_counts()
fn_counts = pd.Series([p for row in comparison['false_negatives'] for p in row]).value_counts()

print("\nTop False Positives (Over-predicted products):")
print(fp_counts.head(10))

print("\nTop False Negatives (Missed products):")
print(fn_counts.head(10))


Top False Positives (Over-predicted products):
ind_nomina_ult1      883144
ind_nom_pens_ult1    878605
ind_cno_fin_ult1     858392
ind_ecue_fin_ult1    855608
ind_ctop_fin_ult1    830864
ind_recibo_ult1      818403
ind_cco_fin_ult1     369838
dtype: int64

Top False Negatives (Missed products):
ind_reca_fin_ult1    45535
ind_tjcr_fin_ult1    34822
ind_ctpp_fin_ult1    33271
ind_dela_fin_ult1    31217
ind_valo_fin_ult1    21426
ind_fond_fin_ult1    14687
ind_ctma_fin_ult1     8097
ind_ctju_fin_ult1     7581
ind_plan_fin_ult1     7359
ind_hip_fin_ult1      4528
dtype: int64


## Item based Collaborative Filtering

This model finds similarities between products based on co-occurrence patterns - if many users who have Product A also have Product B, they are considered similar.

Algorithm:
1. Build user-product interaction matrix (users × products)
2. Calculate cosine similarity between all product pairs
3. For each user:
   - Find products they currently own
   - Sum similarity scores from owned products to all other products
   - Recommend top 7 products with highest aggregate similarity

#### Similarity between products i and j:
- similarity(i,j) = cos(θ) = (A·B)/(||A|| × ||B||)

#### Recommendation score for product p:
- score(p) = Σ similarity(owned_product, p) for all owned products

In [11]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from collections import defaultdict

# Create user-item interaction matrix from training data
def create_interaction_matrix(train_df, product_cols):
    """
    Create sparse matrix of user-product interactions
    """
    # Get unique users
    users = train_df['ncodpers'].unique()
    user_to_idx = {user: idx for idx, user in enumerate(users)}
    
    # Create product to index mapping
    product_to_idx = {prod: idx for idx, prod in enumerate(product_cols)}
    
    # Aggregate interactions per user (taking max to handle multiple months)
    user_products = train_df.groupby('ncodpers')[product_cols].max()
    
    # Create sparse matrix
    row_indices = []
    col_indices = []
    data = []
    
    for user in user_products.index:
        if user in user_to_idx:
            user_idx = user_to_idx[user]
            for prod in product_cols:
                if user_products.loc[user, prod] == 1:
                    row_indices.append(user_idx)
                    col_indices.append(product_to_idx[prod])
                    data.append(1)
    
    interaction_matrix = csr_matrix(
        (data, (row_indices, col_indices)), 
        shape=(len(users), len(product_cols))
    )
    
    return interaction_matrix, user_to_idx, product_to_idx

# Build item similarity matrix
interaction_matrix, user_to_idx, product_to_idx = create_interaction_matrix(train_df, product_cols)

# Compute item-item similarity (transpose for item-based)
item_matrix = interaction_matrix.T
item_similarity = cosine_similarity(item_matrix, dense_output=False)

# Convert to dense for easier manipulation (if memory allows)
item_similarity_df = pd.DataFrame(
    item_similarity.toarray(),
    index=product_cols,
    columns=product_cols
)

print(f"Item similarity matrix shape: {item_similarity_df.shape}")
print(f"Sparsity: {(interaction_matrix.nnz / (interaction_matrix.shape[0] * interaction_matrix.shape[1])):.4f}")

Item similarity matrix shape: (24, 24)
Sparsity: 0.0646


In [ ]:
def get_item_based_recommendations(user_history, item_similarity_df, n_recs=7, exclude_owned=True):
    """
    Generate recommendations based on items the user already has
    """
    # If user has no history, return empty list
    if not user_history:
        return []
    
    # Calculate scores for all products
    scores = {}
    
    for candidate_item in item_similarity_df.columns:
        # Skip if user already owns this product
        if exclude_owned and candidate_item in user_history:
            continue
            
        # Sum similarities from all owned items
        score = 0
        for owned_item in user_history:
            if owned_item in item_similarity_df.index:
                score += item_similarity_df.loc[owned_item, candidate_item]
        
        scores[candidate_item] = score
    
    # Sort and get top N
    sorted_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    recommendations = [item for item, score in sorted_items[:n_recs]]
    
    return recommendations

# Generate predictions for evaluation set
cf_predictions = []

for user_id in eval_users:
    # Get user's current products (from last available month in training)
    user_train_data = train_df[train_df['ncodpers'] == user_id]
    
    if len(user_train_data) > 0:
        # Get most recent month's products
        latest_products = user_train_data.iloc[-1][product_cols]
        owned_products = [prod for prod in product_cols if latest_products[prod] == 1]
        
        # Generate recommendations
        recs = get_item_based_recommendations(
            owned_products, 
            item_similarity_df, 
            n_recs=7
        )
    else:
        # Cold start: fallback to popular items
        recs = top_7_products
    
    cf_predictions.append(recs)

# Create predictions dataframe
cf_preds_df = pd.DataFrame({
    'ncodpers': eval_users,
    'cf_predictions': cf_predictions,
    'cf_predictions_str': [' '.join(preds) for preds in cf_predictions]
})

# Calculate MAP@7 for CF
cf_map_score = mapk(eval_truth, cf_preds_df['cf_predictions_str'].tolist())
print(f"\nItem-Based CF MAP@7: {cf_map_score:.4f}")
print(f"Baseline MAP@7: {map_score:.4f}")
print(f"Improvement: {((cf_map_score - map_score) / map_score * 100):.2f}%")


Item-Based CF MAP@7: 0.0898
Baseline MAP@7: 0.5937
Improvement: -84.87%


### Evaluation on test data from Kaggle

In [14]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

def generate_cf_predictions_optimized(test_users, train_df, product_cols,
                                      item_similarity_df, top_7_products):
    """
    Ultra-fast CF prediction using matrix operations
    """
    print("Preparing data structures...")
    
    # Get April 2016 data (last known state before May predictions)
    april_data = train_df[train_df['fecha_dato'] == '2016-04-28']
    april_data_indexed = april_data.set_index('ncodpers')
    april_users_set = set(april_data_indexed.index)
    
    # Create mappings
    product_to_idx = {prod: idx for idx, prod in enumerate(product_cols)}
    idx_to_product = {idx: prod for idx, prod in enumerate(product_cols)}
    
    # Pre-convert similarity matrix to numpy for faster access
    sim_matrix = item_similarity_df.values
    
    # Prepare baseline for cold-start users
    baseline_pred = ' '.join(top_7_products)
    
    print(f"Processing {len(test_users)} users...")
    print(f"Users with history: {len(april_users_set & set(test_users))}")
    
    # Process all users with vectorized operations where possible
    predictions = []
    
    # Group users by whether they have history
    users_with_history = []
    users_without_history = []
    
    for user_id in test_users:
        if user_id in april_users_set:
            users_with_history.append(user_id)
        else:
            users_without_history.append(user_id)
    
    print(f"Users with history: {len(users_with_history)}")
    print(f"Users without history: {len(users_without_history)}")
    
    # Process users with history in batches
    batch_size = 5000
    user_pred_map = {}
    
    for i in range(0, len(users_with_history), batch_size):
        batch_users = users_with_history[i:i+batch_size]
        
        # Get product ownership for batch
        batch_data = april_data_indexed.loc[batch_users][product_cols].values
        
        # Compute recommendations using matrix multiplication
        # Shape: (batch_size, n_products) @ (n_products, n_products) = (batch_size, n_products)
        recommendation_scores = batch_data @ sim_matrix
        
        # Mask out already owned products
        recommendation_scores[batch_data == 1] = -np.inf
        
        # Generate predictions for batch
        for j, user_id in enumerate(batch_users):
            user_scores = recommendation_scores[j]
            
            # Get top 7 products
            top_indices = np.argsort(user_scores)[-7:][::-1]
            
            # Filter valid recommendations (score > 0)
            valid_recs = [idx_to_product[idx] for idx in top_indices 
                         if user_scores[idx] > 0]
            
            if len(valid_recs) >= 7:
                pred = ' '.join(valid_recs[:7])
            elif len(valid_recs) > 0:
                # Pad with popular products not already recommended
                remaining = [p for p in top_7_products if p not in valid_recs]
                pred = ' '.join(valid_recs + remaining[:7-len(valid_recs)])
            else:
                pred = baseline_pred
            
            user_pred_map[user_id] = pred
        
        if (i + batch_size) % 50000 == 0:
            print(f"Processed {min(i + batch_size, len(users_with_history))}/{len(users_with_history)} users with history")
    
    # Assign baseline to users without history
    for user_id in users_without_history:
        user_pred_map[user_id] = baseline_pred
    
    # Create final predictions list in original order
    predictions = [user_pred_map[user_id] for user_id in test_users]
    
    print("Prediction generation complete!")
    
    return predictions

In [16]:
# Load test data
test_df = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/test_ver2.csv')  # Update path as needed
test_users = test_df['ncodpers'].values

print(f"Total test users: {len(test_users)}")

# Generate CF predictions using batch processing
cf_predictions_batch = generate_cf_predictions_optimized(
    test_users, 
    df_train,  # Your training data
    product_cols,  # Your product columns
    item_similarity_df,  # Your similarity matrix
    top_7_products  # Your top 7 popular products
)

# Create submission file
cf_submission_batch = pd.DataFrame({
    'ncodpers': test_users,
    'added_products': cf_predictions_batch
})

# Save submission
cf_submission_batch.to_csv('cf_submission_batch.csv', index=False)

print("Submission saved to cf_submission_batch.csv")
print(f"Shape: {cf_submission_batch.shape}")

# Quick validation
unique_products = set(' '.join(cf_predictions_batch).split())
print(f"Unique products recommended: {len(unique_products)}")
print("\nFirst 3 predictions:")
for i in range(3):
    print(f"User {test_users[i]}: {cf_predictions_batch[i]}")

/var/folders/jl/5_rcdjz121v7rnhqrmsz6df80000gn/T/ipykernel_1886/3810902245.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/test_ver2.csv')  # Update path as needed


Total test users: 929615
Preparing data structures...
Processing 929615 users...
Users with history: 925252
Users with history: 925252
Users without history: 4363
Processed 50000/925252 users with history
Processed 100000/925252 users with history
Processed 150000/925252 users with history
Processed 200000/925252 users with history
Processed 250000/925252 users with history
Processed 300000/925252 users with history
Processed 350000/925252 users with history
Processed 400000/925252 users with history
Processed 450000/925252 users with history
Processed 500000/925252 users with history
Processed 550000/925252 users with history
Processed 600000/925252 users with history
Processed 650000/925252 users with history
Processed 700000/925252 users with history
Processed 750000/925252 users with history
Processed 800000/925252 users with history
Processed 850000/925252 users with history
Processed 900000/925252 users with history
Prediction generation complete!
Submission saved to cf_submissio

#### Key Characteristics

- Personalization: Recommendations based on user's current product portfolio
- Sparsity: 6.46% density in interaction matrix
- Performance: MAP@7 = 0.0898 (local), 0.014 (Kaggle)

#### Why It Underperformed

Sparse similarity matrix: Many products lack sufficient co-occurrence data
Temporal aggregation issue: Using .max() across all months may include future information
Poor cold start handling: Users with few/no products get weak recommendations

## Hybrid Approach (Item based CF + Popularity)

**How It Works**

The hybrid model combines popularity-based and collaborative filtering scores using a weighted average, controlled by parameter α (alpha).

Algorithm:
1. Calculate popularity score for each product (normalized 0-1)
2. Calculate CF similarity scores for each product (normalized 0-1)
3. Combine scores: hybrid_score = α × CF_score + (1-α) × popularity_score
4. Mask out products user already owns
5. Recommend top 7 products with highest hybrid scores

#### For each product p and user u:
- popularity_score(p) = count(p) / max_count

- cf_score(u,p) = Σ similarity(owned, p) / max_similarity

- hybrid_score(u,p) = α × cf_score(u,p) + (1-α) × popularity_score(p)

#####

- #### Where α ∈ [0,1]:
- #### α = 0: Pure popularity (no personalization)
- #### α = 0.3: 30% CF, 70% popularity (balanced)
- #### α = 1: Pure CF (maximum personalization)

In [19]:
def generate_hybrid_predictions_fast(users, train_df, product_cols, 
                                     item_similarity_df, top_7_products,
                                     alpha=0.3, reference_date='2016-04-28'):
    """
    Fast hybrid model using vectorized operations
    Alpha: weight for CF (0.3 = 30% CF, 70% popularity)
    reference_date: date to use for user's last known products
    """
    print(f"Generating hybrid predictions with alpha={alpha} (CF: {alpha:.0%}, Popularity: {(1-alpha):.0%})")
    
    # Pre-compute popularity scores
    product_popularity = train_df[product_cols].sum()
    pop_scores = product_popularity / product_popularity.max()
    pop_array = pop_scores.values  # Convert to numpy for speed
    
    # Get reference data (April for test, or last month for validation)
    reference_data = train_df[train_df['fecha_dato'] == reference_date]
    reference_data_indexed = reference_data.set_index('ncodpers')
    reference_users_set = set(reference_data_indexed.index)
    
    # Pre-compute similarity matrix as numpy
    sim_matrix = item_similarity_df.values
    
    # Baseline prediction
    baseline_pred = ' '.join(top_7_products)
    
    # Process users in batches
    predictions = []
    batch_size = 10000
    users_with_history = [u for u in users if u in reference_users_set]
    users_without_history = [u for u in users if u not in reference_users_set]
    
    print(f"Users with history: {len(users_with_history)}, without: {len(users_without_history)}")
    
    # Create user prediction map
    user_pred_map = {}
    
    # Process users with history in batches
    for i in range(0, len(users_with_history), batch_size):
        batch_users = users_with_history[i:i+batch_size]
        
        # Get batch data
        batch_data = reference_data_indexed.loc[batch_users][product_cols].values
        
        # Compute CF scores via matrix multiplication
        cf_scores_batch = batch_data @ sim_matrix
        
        # Normalize CF scores per user
        cf_max = np.maximum(cf_scores_batch.max(axis=1, keepdims=True), 1e-10)
        cf_scores_normalized = cf_scores_batch / cf_max
        
        # Combine with popularity scores
        hybrid_scores = alpha * cf_scores_normalized + (1 - alpha) * pop_array
        
        # Mask owned products
        hybrid_scores[batch_data == 1] = -np.inf
        
        # Get top 7 for each user
        for j, user_id in enumerate(batch_users):
            top_indices = np.argsort(hybrid_scores[j])[-7:][::-1]
            top_products = [product_cols[idx] for idx in top_indices]
            user_pred_map[user_id] = ' '.join(top_products)
        
        if (i + batch_size) % 100000 == 0:
            print(f"Processed {min(i + batch_size, len(users_with_history))}/{len(users_with_history)}")
    
    # Assign baseline to users without history
    for user_id in users_without_history:
        user_pred_map[user_id] = baseline_pred
    
    # Create final predictions in original order
    predictions = [user_pred_map[user_id] for user_id in users]
    
    return predictions

In [18]:
def test_alpha_values(test_users_sample, train_df, product_cols, 
                      item_similarity_df, top_7_products):
    """
    Quick test of different alpha values on a sample
    """
    sample_size = min(10000, len(test_users_sample))
    sample_users = test_users_sample[:sample_size]
    
    alphas = [0.1, 0.2, 0.3, 0.4, 0.5]
    results = {}
    
    for alpha in alphas:
        print(f"\nTesting alpha={alpha}")
        preds = generate_hybrid_predictions_fast(
            sample_users, train_df, product_cols,
            item_similarity_df, top_7_products, alpha
        )
        
        # Calculate diversity
        unique_products = len(set(' '.join(preds).split()))
        results[alpha] = unique_products
        print(f"  Unique products: {unique_products}")
    
    return results


In [20]:
def validate_on_eval_set(train_df, eval_df, product_cols, item_similarity_df, 
                         top_7_products, alphas=[0.1, 0.2, 0.3, 0.4, 0.5]):
    """
    Test different alpha values on the evaluation set
    """
    # Get eval users and ground truth
    eval_users = eval_df['ncodpers'].unique()
    
    # Build ground truth
    eval_truth = []
    for _, row in eval_df.groupby('ncodpers'):
        added = [prod for prod in product_cols if row[prod].values[0] == 1]
        eval_truth.append(added)
    
    # For validation, we need to use April data to predict May
    # But train only on data before May
    train_before_may = train_df[train_df['fecha_dato'] < '2016-05-01']
    
    results = {}
    
    for alpha in alphas:
        print(f"\n{'='*50}")
        print(f"Testing alpha={alpha}")
        
        # Generate predictions with this alpha
        predictions = generate_hybrid_predictions_fast(
            eval_users, 
            train_before_may,
            product_cols,
            item_similarity_df,
            top_7_products,
            alpha=alpha,
            reference_date='2016-04-28'  # Use April to predict May
        )
        
        # Calculate MAP@7
        from collections import namedtuple
        
        def apk(actual, predicted, k=7):
            if len(predicted) > k:
                predicted = predicted[:k]
            if not actual:
                return 0
            score = 0.0
            num_hits = 0.0
            for i, p in enumerate(predicted):
                if p in actual and p not in predicted[:i]:
                    num_hits += 1.0
                    score += num_hits / (i+1.0)
            return score / min(len(actual), k)
        
        def mapk(actual_list, predicted_list, k=7):
            return np.mean([apk(a, p.split(), k) for a, p in zip(actual_list, predicted_list)])
        
        map_score = mapk(eval_truth, predictions)
        
        # Calculate diversity
        unique_products = len(set(' '.join(predictions).split()))
        
        results[alpha] = {
            'MAP@7': map_score,
            'unique_products': unique_products
        }
        
        print(f"  MAP@7: {map_score:.4f}")
        print(f"  Unique products: {unique_products}")
        
        # Compare with baseline
        if 'map_score' in locals() or 'map_score' in globals():
            baseline_map = 0.5937  # Your baseline score
            improvement = ((map_score - baseline_map) / baseline_map) * 100
            print(f"  vs Baseline: {improvement:+.2f}%")
    
    # Find best alpha
    best_alpha = max(results, key=lambda x: results[x]['MAP@7'])
    print(f"\n{'='*50}")
    print(f"BEST ALPHA: {best_alpha} with MAP@7: {results[best_alpha]['MAP@7']:.4f}")
    
    return results, best_alpha


In [21]:
# Validate on eval set to find best alpha
print("Validating hybrid model on eval set...")
results, best_alpha = validate_on_eval_set(
    train_df, 
    eval_df, 
    product_cols, 
    item_similarity_df, 
    top_7_products,
    alphas=[0.1, 0.2, 0.3, 0.4, 0.5]
)

print("\nSummary of results:")
for alpha, metrics in results.items():
    print(f"Alpha {alpha}: MAP@7={metrics['MAP@7']:.4f}, Products={metrics['unique_products']}")

Validating hybrid model on eval set...

Testing alpha=0.1
Generating hybrid predictions with alpha=0.1 (CF: 10%, Popularity: 90%)
Users with history: 926663, without: 4790
Processed 100000/926663
Processed 200000/926663
Processed 300000/926663
Processed 400000/926663
Processed 500000/926663
Processed 600000/926663
Processed 700000/926663
Processed 800000/926663
Processed 900000/926663
  MAP@7: 0.2928
  Unique products: 20
  vs Baseline: -50.68%

Testing alpha=0.2
Generating hybrid predictions with alpha=0.2 (CF: 20%, Popularity: 80%)
Users with history: 926663, without: 4790
Processed 100000/926663
Processed 200000/926663
Processed 300000/926663
Processed 400000/926663
Processed 500000/926663
Processed 600000/926663
Processed 700000/926663
Processed 800000/926663
Processed 900000/926663
  MAP@7: 0.2908
  Unique products: 21
  vs Baseline: -51.02%

Testing alpha=0.3
Generating hybrid predictions with alpha=0.3 (CF: 30%, Popularity: 70%)
Users with history: 926663, without: 4790
Processe

#### Key Observations

All hybrid configurations underperform: Every alpha value produces MAP@7 around 0.29, which is ~50% worse than your baseline (0.5937)

Inverse relationship: As CF weight increases (higher alpha), performance decreases

This confirms CF is actively hurting predictions


Limited product diversity: Only 20-21 unique products being recommended across all users

##### What's Going Wrong? 

The discrepancy between the baseline score (0.5937) and hybrid scores (~0.29) suggests a fundamental implementation issue:

##### Validation vs Test Mismatch:

- Your baseline MAP@7 of 0.5937 seems unusually high for local validation
- Kaggle baseline was 0.017, which is more realistic
- This suggests the validation setup might be different from how you calculated the baseline


##### Possible Data Leakage:

The baseline might have been calculated differently (possibly using future information)
Or the eval_truth calculation might be different

#### Error Anlaysis or Debugging

In [23]:
# 1. Verify baseline calculation on same eval set
print(f"Eval users count: {len(eval_users)}")
print(f"Baseline predictions count: {len(baseline_preds)}")

# 2. Check a few predictions manually
# First, let's get the predictions from the hybrid model
# We need to run the hybrid model again to store predictions
hybrid_test_predictions = generate_hybrid_predictions_fast(
    eval_users, 
    train_df,
    product_cols,
    item_similarity_df,
    top_7_products,
    alpha=0.1,
    reference_date='2016-04-28'
)

for i in range(3):
    user = eval_users[i]
    print(f"\nUser {user}:")
    print(f"  Baseline pred: {baseline_preds[baseline_preds['ncodpers']==user]['added_products'].values[0][:50]}...")
    print(f"  Hybrid pred: {hybrid_test_predictions[i][:50]}...")
    print(f"  Actual products: {eval_truth[i]}")

# 3. Recalculate baseline MAP@7 using same function
baseline_map_recalc = mapk(eval_truth, baseline_preds['added_products'].tolist())
print(f"\nRecalculated baseline MAP@7: {baseline_map_recalc:.4f}")

# 4. Also test pure popularity (alpha=0) which should match baseline
pure_popularity_predictions = generate_hybrid_predictions_fast(
    eval_users, 
    train_df,
    product_cols,
    item_similarity_df,
    top_7_products,
    alpha=0.0,  # Pure popularity
    reference_date='2016-04-28'
)

pure_pop_map = mapk(eval_truth, pure_popularity_predictions)
print(f"Pure popularity (alpha=0) MAP@7: {pure_pop_map:.4f}")
print(f"Should match baseline: {baseline_map_recalc:.4f}")

# 5. Check if predictions are identical
matches = sum(1 for i in range(len(eval_users)) 
              if baseline_preds['added_products'].iloc[i] == pure_popularity_predictions[i])
print(f"\nPredictions matching between baseline and alpha=0: {matches}/{len(eval_users)}")

Eval users count: 931453
Baseline predictions count: 931453
Generating hybrid predictions with alpha=0.1 (CF: 10%, Popularity: 90%)
Users with history: 926663, without: 4790
Processed 100000/926663
Processed 200000/926663
Processed 300000/926663
Processed 400000/926663
Processed 500000/926663
Processed 600000/926663
Processed 700000/926663
Processed 800000/926663
Processed 900000/926663

User 657640:
  Baseline pred: ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_ult1...
  Hybrid pred: ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_ult1...
  Actual products: ['ind_cco_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1']

User 657788:
  Baseline pred: ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_ult1...
  Hybrid pred: ind_recibo_ult1 ind_ctop_fin_ult1 ind_ecue_fin_ult...
  Actual products: ['ind_cno_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_ecue_fin_ult1', 'ind_plan_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

User 657795:
  

##### Critical Issue Found: The Hybrid Model Implementation Has a Bug

The output reveals the problem clearly:

Key Discovery

- Baseline MAP@7: 0.5937 (confirmed)
- Pure popularity (alpha=0) MAP@7: 0.2964 (should be identical to baseline)
- Matching predictions: Only 265,150/931,453 (28.5%) match between baseline and alpha=0

This proves there's a bug in the hybrid model implementation, not a conceptual issue.

**The Problem**

When alpha=0 (pure popularity), the hybrid model should produce exactly the same predictions as your baseline, but it doesn't. 

The hybrid model is excluding products the user already owns, but your baseline doesn't do this! Your evaluation is likely measuring which products users have in May that they didn't have in April (new acquisitions), but the hybrid model is excluding all products they already own.

The Fix

We need to modify the hybrid model to NOT exclude owned products when making predictions

In [24]:
def generate_hybrid_predictions_fixed(users, train_df, product_cols, 
                                      item_similarity_df, top_7_products,
                                      alpha=0.3, reference_date='2016-04-28'):
    """
    Fixed hybrid model - doesn't exclude owned products
    """
    print(f"Generating hybrid predictions with alpha={alpha} (CF: {alpha:.0%}, Popularity: {(1-alpha):.0%})")
    
    # Pre-compute popularity scores
    product_popularity = train_df[product_cols].sum()
    pop_scores = product_popularity / product_popularity.max()
    pop_array = pop_scores.values
    
    # Get reference data
    reference_data = train_df[train_df['fecha_dato'] == reference_date]
    reference_data_indexed = reference_data.set_index('ncodpers')
    reference_users_set = set(reference_data_indexed.index)
    
    # Pre-compute similarity matrix
    sim_matrix = item_similarity_df.values
    
    # Baseline prediction
    baseline_pred = ' '.join(top_7_products)
    
    # If alpha is 0, just return popularity for everyone
    if alpha == 0:
        return [baseline_pred] * len(users)
    
    # Process users
    predictions = []
    batch_size = 10000
    users_with_history = [u for u in users if u in reference_users_set]
    users_without_history = [u for u in users if u not in reference_users_set]
    
    print(f"Users with history: {len(users_with_history)}, without: {len(users_without_history)}")
    
    user_pred_map = {}
    
    # Process users with history
    for i in range(0, len(users_with_history), batch_size):
        batch_users = users_with_history[i:i+batch_size]
        batch_data = reference_data_indexed.loc[batch_users][product_cols].values
        
        # Compute CF scores
        cf_scores_batch = batch_data @ sim_matrix
        cf_max = np.maximum(cf_scores_batch.max(axis=1, keepdims=True), 1e-10)
        cf_scores_normalized = cf_scores_batch / cf_max
        
        # Combine with popularity
        hybrid_scores = alpha * cf_scores_normalized + (1 - alpha) * pop_array
        
        # DON'T mask owned products - we want to recommend all top products
        # hybrid_scores[batch_data == 1] = -np.inf  # REMOVED THIS LINE
        
        # Get top 7 for each user
        for j, user_id in enumerate(batch_users):
            top_indices = np.argsort(hybrid_scores[j])[-7:][::-1]
            top_products = [product_cols[idx] for idx in top_indices]
            user_pred_map[user_id] = ' '.join(top_products)
        
        if (i + batch_size) % 100000 == 0:
            print(f"Processed {min(i + batch_size, len(users_with_history))}/{len(users_with_history)}")
    
    # Users without history get baseline
    for user_id in users_without_history:
        user_pred_map[user_id] = baseline_pred
    
    # Create final predictions
    predictions = [user_pred_map[user_id] for user_id in users]
    
    return predictions

# Test the fixed version
fixed_predictions = generate_hybrid_predictions_fixed(
    eval_users, 
    train_df,
    product_cols,
    item_similarity_df,
    top_7_products,
    alpha=0.0  # Pure popularity
)

fixed_map = mapk(eval_truth, fixed_predictions)
print(f"Fixed model with alpha=0 MAP@7: {fixed_map:.4f}")
print(f"Should now match baseline: {baseline_map_recalc:.4f}")

Generating hybrid predictions with alpha=0.0 (CF: 0%, Popularity: 100%)
Fixed model with alpha=0 MAP@7: 0.5937
Should now match baseline: 0.5937


In [25]:
# Test different alpha values with the FIXED model
def validate_fixed_hybrid(train_df, eval_df, product_cols, item_similarity_df, 
                         top_7_products, eval_users, eval_truth,
                         alphas=[0.0, 0.05, 0.1, 0.15, 0.2, 0.3]):
    """
    Test the fixed hybrid model with different alpha values
    """
    results = {}
    
    for alpha in alphas:
        print(f"\nTesting alpha={alpha}")
        
        predictions = generate_hybrid_predictions_fixed(
            eval_users, 
            train_df,
            product_cols,
            item_similarity_df,
            top_7_products,
            alpha=alpha,
            reference_date='2016-04-28'
        )
        
        map_score = mapk(eval_truth, predictions)
        unique_products = len(set(' '.join(predictions).split()))
        
        results[alpha] = {
            'MAP@7': map_score,
            'unique_products': unique_products
        }
        
        print(f"  MAP@7: {map_score:.4f}")
        print(f"  Unique products: {unique_products}")
        
        # Show improvement/degradation from baseline
        baseline_map = 0.5937
        change = ((map_score - baseline_map) / baseline_map) * 100
        print(f"  vs Baseline: {change:+.2f}%")
    
    return results

# Run validation
results_fixed = validate_fixed_hybrid(
    train_df, eval_df, product_cols, item_similarity_df, 
    top_7_products, eval_users, eval_truth,
    alphas=[0.0, 0.05, 0.1, 0.15, 0.2, 0.3]
)

# Find best alpha
best_alpha_fixed = max(results_fixed, key=lambda x: results_fixed[x]['MAP@7'])
print(f"\n{'='*50}")
print(f"BEST ALPHA: {best_alpha_fixed} with MAP@7: {results_fixed[best_alpha_fixed]['MAP@7']:.4f}")

# Summary
print("\nSummary of Fixed Model Results:")
for alpha, metrics in results_fixed.items():
    improvement = ((metrics['MAP@7'] - 0.5937) / 0.5937) * 100
    print(f"Alpha {alpha:4.2f}: MAP@7={metrics['MAP@7']:.4f} ({improvement:+.2f}%), Products={metrics['unique_products']}")


Testing alpha=0.0
Generating hybrid predictions with alpha=0.0 (CF: 0%, Popularity: 100%)
  MAP@7: 0.5937
  Unique products: 7
  vs Baseline: -0.00%

Testing alpha=0.05
Generating hybrid predictions with alpha=0.05 (CF: 5%, Popularity: 95%)
Users with history: 926663, without: 4790
Processed 100000/926663
Processed 200000/926663
Processed 300000/926663
Processed 400000/926663
Processed 500000/926663
Processed 600000/926663
Processed 700000/926663
Processed 800000/926663
Processed 900000/926663
  MAP@7: 0.5916
  Unique products: 12
  vs Baseline: -0.35%

Testing alpha=0.1
Generating hybrid predictions with alpha=0.1 (CF: 10%, Popularity: 90%)
Users with history: 926663, without: 4790
Processed 100000/926663
Processed 200000/926663
Processed 300000/926663
Processed 400000/926663
Processed 500000/926663
Processed 600000/926663
Processed 700000/926663
Processed 800000/926663
Processed 900000/926663
  MAP@7: 0.5908
  Unique products: 23
  vs Baseline: -0.49%

Testing alpha=0.15
Generating 

In [26]:
# Generate test predictions with alpha=0.05
chosen_alpha = 0.05  # Minimal CF influence

print(f"\nGenerating test predictions with alpha={chosen_alpha}")
test_predictions = generate_hybrid_predictions_fixed(
    test_users,
    df_train,
    product_cols,
    item_similarity_df,
    top_7_products,
    alpha=chosen_alpha,
    reference_date='2016-04-28'
)

submission = pd.DataFrame({
    'ncodpers': test_users,
    'added_products': test_predictions
})

submission.to_csv('hybrid_alpha005.csv', index=False)
print("Saved to hybrid_alpha005.csv")


Generating test predictions with alpha=0.05
Generating hybrid predictions with alpha=0.05 (CF: 5%, Popularity: 95%)
Users with history: 925252, without: 4363
Processed 100000/925252
Processed 200000/925252
Processed 300000/925252
Processed 400000/925252
Processed 500000/925252
Processed 600000/925252
Processed 700000/925252
Processed 800000/925252
Processed 900000/925252
Saved to hybrid_alpha005.csv


## Matrix Factorization

### What is Matrix Factorization?

Matrix Factorization is like finding the "hidden reasons" why users choose certain products. Instead of looking at individual product similarities, it discovers underlying patterns.

Analogy: Imagine Netflix movies. Instead of comparing movies directly, Matrix Factorization might discover 

hidden factors like:

- Factor 1: "Action vs Romance"
- Factor 2: "Indie vs Blockbuster"
- Factor 3: "Comedy vs Drama"

For banking, these factors might represent things like:

- Factor 1: "Basic vs Premium products"
- Factor 2: "Savings vs Credit products"
- Factor 3: "Digital vs Traditional banking

### How The Algorithm Works

MATRIX FACTORIZATION ALGORITHM:

1. START with User-Product Matrix (931K users × 24 products)
   - 1 = user has product
   - 0 = user doesn't have product

2. DECOMPOSE into smaller matrices:
   Original Matrix ≈ User_Factors × Product_Factors
   (931K × 24)   ≈  (931K × 15)  ×  (15 × 24)
   
3. DISCOVER latent features:
   - Each user gets 15 numbers (their preferences for each hidden factor)
   - Each product gets 15 numbers (how much it relates to each factor)

4. PREDICT scores:
   Score(user, product) = user_factors · product_factors
   
5. RECOMMEND top 7 products with highest scores

### Building Matrix

*create_matrix_factorization_model(train_df, product_cols, method='svd', n_factors=15)*

What it does:

- Takes all user-product interactions
- Creates a sparse matrix (mostly zeros)
- Uses SVD (Singular Value Decomposition) to find 15 hidden factors
- Returns two smaller matrices: user preferences and product characteristics

In [27]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.decomposition import TruncatedSVD, NMF

def create_matrix_factorization_model(train_df, product_cols, method='svd', n_factors=10):
    """
    Create matrix factorization model using SVD or NMF
    
    Parameters:
    -----------
    method: 'svd' or 'nmf'
    n_factors: number of latent factors
    """
    print(f"Building {method.upper()} matrix factorization model with {n_factors} factors...")
    
    # Create user-product matrix
    users = train_df['ncodpers'].unique()
    user_to_idx = {user: idx for idx, user in enumerate(users)}
    product_to_idx = {prod: idx for idx, prod in enumerate(product_cols)}
    
    # Aggregate by taking max (user ever had product)
    user_products = train_df.groupby('ncodpers')[product_cols].max()
    
    # Create sparse matrix
    n_users = len(users)
    n_products = len(product_cols)
    
    print(f"Matrix dimensions: {n_users} users x {n_products} products")
    
    # Build sparse matrix
    row_indices = []
    col_indices = []
    data = []
    
    for user in user_products.index:
        if user in user_to_idx:
            user_idx = user_to_idx[user]
            for prod in product_cols:
                if user_products.loc[user, prod] == 1:
                    row_indices.append(user_idx)
                    col_indices.append(product_to_idx[prod])
                    data.append(1)
    
    sparse_matrix = csr_matrix((data, (row_indices, col_indices)), 
                               shape=(n_users, n_products))
    
    print(f"Sparsity: {len(data) / (n_users * n_products):.4f}")
    
    if method == 'svd':
        # Use Truncated SVD for sparse matrices
        model = TruncatedSVD(n_components=n_factors, random_state=42)
        user_factors = model.fit_transform(sparse_matrix)
        product_factors = model.components_.T
        
    elif method == 'nmf':
        # Non-negative Matrix Factorization
        model = NMF(n_components=n_factors, init='random', random_state=42, max_iter=200)
        user_factors = model.fit_transform(sparse_matrix)
        product_factors = model.components_.T
    
    else:
        # Traditional SVD (slower, for smaller matrices)
        U, sigma, Vt = svds(sparse_matrix.asfptype(), k=n_factors)
        user_factors = U * sigma
        product_factors = Vt.T
    
    print(f"User factors shape: {user_factors.shape}")
    print(f"Product factors shape: {product_factors.shape}")
    
    return {
        'user_factors': user_factors,
        'product_factors': product_factors,
        'user_to_idx': user_to_idx,
        'product_to_idx': product_to_idx,
        'idx_to_product': {v: k for k, v in product_to_idx.items()},
        'users': users,
        'model': model if method in ['svd', 'nmf'] else None
    }


### Generate Predictions

*generate_mf_predictions(users, train_df, product_cols, mf_model, top_7_products)*

What it does:

- For each user: multiplies their 15 factor scores with all products' 15 factors
- This gives a score for every product
- Sorts and picks top 7 highest scoring products
- Users not in training get popular products as fallback

In [28]:
def generate_mf_predictions(users_to_predict, train_df, product_cols, 
                           mf_model, top_7_products, reference_date='2016-04-28'):
    """
    Generate predictions using matrix factorization
    """
    print("Generating MF predictions...")
    
    # Get reference data
    reference_data = train_df[train_df['fecha_dato'] == reference_date]
    reference_users = set(reference_data['ncodpers'].unique())
    
    # Extract model components
    user_factors = mf_model['user_factors']
    product_factors = mf_model['product_factors']
    user_to_idx = mf_model['user_to_idx']
    idx_to_product = mf_model['idx_to_product']
    
    predictions = []
    baseline_pred = ' '.join(top_7_products)
    
    users_with_factors = 0
    users_without_factors = 0
    
    for user in users_to_predict:
        if user in user_to_idx and user in reference_users:
            users_with_factors += 1
            
            # Get user's latent factors
            user_idx = user_to_idx[user]
            user_vec = user_factors[user_idx]
            
            # Compute scores for all products
            scores = user_vec @ product_factors.T
            
            # Get user's current products to exclude (optional)
            user_data = reference_data[reference_data['ncodpers'] == user]
            if len(user_data) > 0:
                owned_products = user_data.iloc[0][product_cols].values
                # Don't exclude owned products based on previous findings
                # scores[owned_products == 1] = -np.inf
            
            # Get top 7
            top_indices = np.argsort(scores)[-7:][::-1]
            top_products = [idx_to_product[idx] for idx in top_indices]
            pred = ' '.join(top_products)
            
        else:
            users_without_factors += 1
            pred = baseline_pred
        
        predictions.append(pred)
        
        if len(predictions) % 100000 == 0:
            print(f"Processed {len(predictions)}/{len(users_to_predict)} users")
    
    print(f"Users with factors: {users_with_factors}")
    print(f"Users without factors (using baseline): {users_without_factors}")
    
    return predictions

### Hybrid Approach on Matrix Factorization

*hybrid_mf_popularity(users, train_df, product_cols, mf_model, top_7_products, alpha=0.3)*

What it does:

- Combines MF scores with popularity scores
- Formula: final_score = α × MF_score + (1-α) × popularity_score
- α=0.3 means 30% personalization, 70% popularity

In [29]:
def hybrid_mf_popularity(users_to_predict, train_df, product_cols,
                        mf_model, top_7_products, alpha=0.3, reference_date='2016-04-28'):
    """
    Hybrid: Matrix Factorization + Popularity
    """
    print(f"Generating hybrid MF predictions (alpha={alpha}: {alpha:.0%} MF, {(1-alpha):.0%} popularity)")
    
    # Popularity scores
    product_popularity = train_df[product_cols].sum()
    pop_scores = product_popularity / product_popularity.max()
    pop_array = pop_scores.values
    
    # Reference data
    reference_data = train_df[train_df['fecha_dato'] == reference_date]
    reference_users = set(reference_data['ncodpers'].unique())
    
    # Model components
    user_factors = mf_model['user_factors']
    product_factors = mf_model['product_factors']
    user_to_idx = mf_model['user_to_idx']
    idx_to_product = mf_model['idx_to_product']
    
    predictions = []
    baseline_pred = ' '.join(top_7_products)
    
    for user in users_to_predict:
        if user in user_to_idx and user in reference_users:
            # Get MF scores
            user_idx = user_to_idx[user]
            user_vec = user_factors[user_idx]
            mf_scores = user_vec @ product_factors.T
            
            # Normalize MF scores
            if mf_scores.max() > mf_scores.min():
                mf_scores_norm = (mf_scores - mf_scores.min()) / (mf_scores.max() - mf_scores.min())
            else:
                mf_scores_norm = mf_scores
            
            # Combine with popularity
            hybrid_scores = alpha * mf_scores_norm + (1 - alpha) * pop_array
            
            # Get top 7
            top_indices = np.argsort(hybrid_scores)[-7:][::-1]
            top_products = [idx_to_product[idx] for idx in top_indices]
            pred = ' '.join(top_products)
        else:
            pred = baseline_pred
        
        predictions.append(pred)
    
    return predictions

### What it does:

Original Matrix (Sparse, Huge):
Users    | Product1 | Product2 | ... | Product24
---------|----------|----------|-----|----------
User1    |    1     |    0     | ... |    1
User2    |    0     |    1     | ... |    0
...      |   ...    |   ...    | ... |   ...
User931K |    1     |    1     | ... |    0

After Factorization:

User_Factors (Dense, Smaller):        
Users    | F1  | F2  | ... | F15      
---------|-----|-----|-----|-----     
User1    | 0.2 | 0.8 | ... | 0.1      
User2    | 0.9 | 0.1 | ... | 0.7      

Product_Factors:
Products | F1  | F2  | ... | F15
---------|-----|-----|-----|-----
Prod1    | 0.5 | 0.1 | ... | 0.3
Prod2    | 0.2 | 0.9 | ... | 0.4

To predict if User1 should get Product2:

Score = (0.2×0.2) + (0.8×0.9) + ... + (0.1×0.4) = 0.85

**The core idea:** Instead of saying "people who bought A also bought B", Matrix Factorization says "people who like savings products and digital banking tend to choose these specific products."

In [30]:
# 1. Build MF model
mf_model_svd = create_matrix_factorization_model(
    train_df[train_df['fecha_dato'] < '2016-05-01'],  # Train only on data before May
    product_cols,
    method='svd',  # or try 'nmf'
    n_factors=15  # Start with 15 factors
)

# 2. Test pure MF
print("\nValidating MF model on eval set...")
mf_predictions = generate_mf_predictions(
    eval_users,
    train_df,
    product_cols,
    mf_model_svd,
    top_7_products,
    reference_date='2016-04-28'
)

mf_map = mapk(eval_truth, mf_predictions)
print(f"Pure MF MAP@7: {mf_map:.4f}")
print(f"vs Baseline: {((mf_map - 0.5937) / 0.5937 * 100):+.2f}%")

Building SVD matrix factorization model with 15 factors...
Matrix dimensions: 944901 users x 24 products
Sparsity: 0.0646
User factors shape: (944901, 15)
Product factors shape: (24, 15)

Validating MF model on eval set...
Generating MF predictions...
Processed 100000/931453 users
Processed 200000/931453 users
Processed 300000/931453 users
Processed 400000/931453 users
Processed 500000/931453 users
Processed 600000/931453 users
Processed 700000/931453 users
Processed 800000/931453 users
Processed 900000/931453 users
Users with factors: 926663
Users without factors (using baseline): 4790
Pure MF MAP@7: 0.3783
vs Baseline: -36.27%


In [ ]:
# Generate test predictions with alpha=0.05
chosen_alpha = 0.05  # Minimal CF influence

print(f"\nGenerating test predictions with alpha={chosen_alpha}")
test_predictions = generate_hybrid_predictions_fixed(
    test_users,
    df_train,
    product_cols,
    item_similarity_df,
    top_7_products,
    alpha=chosen_alpha,
    reference_date='2016-04-28'
)

submission = pd.DataFrame({
    'ncodpers': test_users,
    'added_products': test_predictions
})

submission.to_csv('hybrid_alpha005.csv', index=False)
print("Saved to hybrid_alpha005.csv")

In [31]:
# Test different alpha values for hybrid MF + Popularity
alphas_to_test = [0.05, 0.1, 0.15, 0.2, 0.3]
results_mf_hybrid = {}

for alpha in alphas_to_test:
    print(f"\nTesting MF hybrid with alpha={alpha}")
    hybrid_mf_preds = hybrid_mf_popularity(
        eval_users,
        train_df,
        product_cols,
        mf_model_svd,
        top_7_products,
        alpha=alpha,
        reference_date='2016-04-28'
    )
    
    score = mapk(eval_truth, hybrid_mf_preds)
    results_mf_hybrid[alpha] = score
    print(f"  MAP@7={score:.4f} ({((score - 0.5937) / 0.5937 * 100):+.2f}% vs baseline)")

# Find best configuration
best_alpha_mf = max(results_mf_hybrid, key=results_mf_hybrid.get)
best_score_mf = results_mf_hybrid[best_alpha_mf]

print(f"\n{'='*50}")
print(f"BEST MF Hybrid: alpha={best_alpha_mf}, MAP@7={best_score_mf:.4f}")
print(f"Improvement over baseline: {((best_score_mf - 0.5937) / 0.5937 * 100):+.2f}%")


Testing MF hybrid with alpha=0.05
Generating hybrid MF predictions (alpha=0.05: 5% MF, 95% popularity)
  MAP@7=0.5927 (-0.17% vs baseline)

Testing MF hybrid with alpha=0.1
Generating hybrid MF predictions (alpha=0.1: 10% MF, 90% popularity)
  MAP@7=0.5910 (-0.45% vs baseline)

Testing MF hybrid with alpha=0.15
Generating hybrid MF predictions (alpha=0.15: 15% MF, 85% popularity)
  MAP@7=0.5884 (-0.89% vs baseline)

Testing MF hybrid with alpha=0.2
Generating hybrid MF predictions (alpha=0.2: 20% MF, 80% popularity)
  MAP@7=0.5878 (-1.00% vs baseline)

Testing MF hybrid with alpha=0.3
Generating hybrid MF predictions (alpha=0.3: 30% MF, 70% popularity)
  MAP@7=0.5877 (-1.01% vs baseline)

BEST MF Hybrid: alpha=0.05, MAP@7=0.5927
Improvement over baseline: -0.17%


In [32]:
# Generate test predictions with MF hybrid
print("Generating test predictions with MF hybrid model...")

# First, rebuild the MF model on ALL training data (not just pre-May)
print("\n1. Rebuilding MF model on full training data...")
mf_model_full = create_matrix_factorization_model(
    df_train,  # Use ALL training data now
    product_cols,
    method='svd',
    n_factors=15
)

# Load test data if needed
if 'test_df' not in locals():
    test_df = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/test_ver2.csv')
    test_users = test_df['ncodpers'].values
    print(f"Loaded {len(test_users)} test users")

# Generate predictions with best alpha from validation (0.05)
best_alpha_mf = 0.05
print(f"\n2. Generating hybrid predictions with alpha={best_alpha_mf}")

test_predictions_mf = hybrid_mf_popularity(
    test_users,
    df_train,
    product_cols,
    mf_model_full,
    top_7_products,
    alpha=best_alpha_mf,
    reference_date='2016-04-28'
)

# Create submission
submission_mf = pd.DataFrame({
    'ncodpers': test_users,
    'added_products': test_predictions_mf
})

# Save submission
filename = f'mf_hybrid_alpha{int(best_alpha_mf*100):02d}.csv'
submission_mf.to_csv(filename, index=False)
print(f"\n3. Saved to {filename}")
print(f"Shape: {submission_mf.shape}")

# Quick stats
unique_products = set(' '.join(test_predictions_mf).split())
print(f"Unique products recommended: {len(unique_products)}")
print(f"\nFirst 3 predictions:")
for i in range(3):
    print(f"User {test_users[i]}: {test_predictions_mf[i][:50]}...")

print(f"\nSubmit {filename} to Kaggle to see if MF adds any value on test set")

Generating test predictions with MF hybrid model...

1. Rebuilding MF model on full training data...
Building SVD matrix factorization model with 15 factors...
Matrix dimensions: 949614 users x 24 products
Sparsity: 0.0649
User factors shape: (949614, 15)
Product factors shape: (24, 15)

2. Generating hybrid predictions with alpha=0.05
Generating hybrid MF predictions (alpha=0.05: 5% MF, 95% popularity)

3. Saved to mf_hybrid_alpha05.csv
Shape: (929615, 2)
Unique products recommended: 12

First 3 predictions:
User 15889: ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_ult1...
User 1170544: ind_cco_fin_ult1 ind_recibo_ult1 ind_ctop_fin_ult1...
User 1170545: ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_ult1...

Submit mf_hybrid_alpha05.csv to Kaggle to see if MF adds any value on test set
